<a href="https://colab.research.google.com/github/asefycom/Python-with-AI/blob/main/3rd-week/L07/L07_create_itinerary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# هفته سوم - درس ۷: ساخت برنامه سفر به چند شهر

با چیزهایی که تا حالا یاد گرفتی تو این درس بناست که برنامه سفر به دور دنیا برای تعطیلات بسازی.

## پیشنیازها

In [ ]:
# ایمپورت‌های لازم
from IPython.display import Markdown
import csv
import pandas as pd

In [ ]:
# تابعی برای نمایش داده ها به صورت جدول
def display_table(data):
    # تبدیل داده ورودی به دیتافریم
    df = pd.DataFrame(data)

    # نمایش دیتافریم به شکل جدول ا‌چ‌تی‌ام‌ال
    display(HTML(df.to_html(index=False)))

In [ ]:
# نصب بسته توگدر
!pip install together

In [ ]:
# فراخوانی توگدر با کلیدی که ازش گرفتیم
# که به صورت امن تو محیط اجرای این دفترچه کولب کار گذاشتیم
from google.colab import userdata
from together import Together

client = Together(api_key=userdata.get('TOGETHER_API_KEY'))

# تابعی برای ارسال پرامپت به مدل زبانی به واسطه توگدر و بعد چاپ جواب
def print_llm_response(prompt):
    response = client.chat.completions.create(
        model="google/gemma-3n-E4B-it",
        # model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
        messages=[
        {
            "role": "user",
            "content": prompt
        }
        ]
    )
    return response.choices[0].message.content

## واکشی برنامه‌ سفر از فایل‌ سی‌اس‌وی

نخست بیا تابعی تعریف کنیم که اطلاعات فایل سی‌اس‌وی رو بخونه و به صورت یک دیکشنری به ما بده.

In [ ]:
def read_csv(file):
    f = open(file, "r")

    csv_reader = csv.DictReader(f)
    data = []
    for row in csv_reader:
        data.append(row)
    f.close()

    return data

حالا از این تابع استفاده کنیم. برنامه‌های سفر رو از فایل

 `itinerary.csv`

 بخونیم و اطلاعاتش رو که به صورت دیکشنری دریافت می‌کنیم به صورت یک جدول نمایش بدیم.

In [ ]:
# Read the itinerary.csv file
itinerary = read_csv("itinerary.csv")

# Display the itinerary
display_table(itinerary)

## واکشی اطلاعات رستوران‌ها از یادداشت‌های غذایی

حالا تابعلی بسازیم که محتوای فایل متنی رو می‌خونه و به صورت یک استرینگ بلند برمی‌گردونه.

In [ ]:
# این تابع رو «رید جورنال» نام گذاشتیم
def read_journal(journal_file):
    f = open(journal_file, "r")
    journal = f.read()
    f.close()

    # بازگشت محتوای یادداشت
    return journal

حالا برای نمونه از این تابع استفاده کنیم و یادداشت شهر سیدنی رو بخونیم.

In [ ]:
journal = read_journal("sydney.txt")

print(journal)

و حالا ... یک پرامپت (سپارش) بنویس که اطلاعات رستوران‌ها و غذای مخصوص هر کدوم رو استخراج و به فرمت سی‌اس‌وی ذخیره کنه.

In [ ]:
# Write the prompt
prompt = f"""Please extract a comprehensive list of the restaurants
and their respective specialties mentioned in the following journal entry.
Ensure that each restaurant name is accurately identified and listed.
Provide your answer in CSV format, ready to save.
Exclude the "```csv" declaration, don't add spaces after the comma, include column headers.

Format:
Restaurant, Specialty
Res_1, Sp_1
...

Journal entry:
{journal}
"""

# Print the prompt
print(print_llm_response(prompt))

از مدل زبانی بخواه تابعی برای ساخت فایل سی اس وی بنویسه.
تابعی نیاز داریم که داده‌هایی که با فرمت سی‌اس‌وی ساختیم در یک فایل ذخیره کنه.

فایل پایین یک نمونه ذخیره شده از خروجی مدل هست:

`Sydney.csv`

فایل رو بخون و به صورت جدول نمایش بده:

In [ ]:
# استفاده از تابعی که پیش‌تر تعریف کردی
sydney_restaurants = read_csv("Sydney.csv")

display_table(sydney_restaurants)

## ساخت برنامه سفر کامل با پیشنهادهایی برای رستوران

بیا داده‌های برنامه  سفر و یادداشت غذا رو کنار هم بگذاریم و یک برنامه کامل برای بازدیداز سیدنی بسازیم.

دقت کن که تو فایل برنامه سفر اطلاعات سیدنی در ایندکس ۶ هستن چون سیدنی هفتمین مقصد سفر هست.

In [ ]:
# انتخاب سیدنی از برنامه سفر
trip_stop = itinerary[6]

حالا همه اطلاعاتی که داری رو داخل متغیرهای جداگانه ذخیره کن: اطلاعات زمان بندی ورود و خروج سیدنی و رستوران‌هاش

In [ ]:
city = trip_stop["City"]
country = trip_stop["Country"]
arrival = trip_stop["Arrival"]
departure = trip_stop["Departure"]
restaurants = sydney_restaurants

سرانجام همه این اطلاعات رو در یک پرامپت به مدل‌زبانی بده تا یک برنامه سفر جامع تهیه کنه.

In [ ]:
# متن پرامپت یا سپارش
prompt = f"""I will visit {city}, {country} from {arrival} to {departure}.
Create a daily itinerary with detailed activities.
Designate times for breakfast, lunch, and dinner.

I want to visit the restaurants listed in the restaurant dictionary
without repeating any place. Make sure to mention the specialty
that I should try at each of them.

Restaurant dictionary:
{restaurants}

"""

response = print_llm_response(prompt)

# چاپ پاسخ مدل به فرمت مارک‌داون
display(Markdown(response))

## ساخت برنامه جامع سفر برای همه شهرها

یک حلقه فور و بعد بوممممم! همه چیز برای همه شهرها تکرار می‌شه.

In [ ]:
# دیکشنری خالی برای ثبت برنامه همه شهرها
detailed_itinerary = {}

 # حلقه تکرار برای واکشی اطلاعات همه مقصدها
for trip_stop in itinerary:
    city = trip_stop["City"]
    country = trip_stop["Country"]
    arrival = trip_stop["Arrival"]
    departure = trip_stop["Departure"]

    rest_dict = read_csv(f"{city}.csv")

    print(f"Creating detailed itinerary for {city}, {country}.")

    prompt = f"""I will visit {city}, {country} from {arrival} to {departure}.
    Create a daily itinerary with detailed activities.
    Designate times for breakfast, lunch, and dinner.

    I want to visit the restaurants listed in the restaurant dictionary without repeating any place.
    Make sure to mention the specialty that I should try at each of them.

    Restaurant dictionary:
    {rest_dict}

    """
    # ذخیره برنامه سفر پیشنهادی برای همه شهرها در دیکشنری
    detailed_itinerary[city] = print(get_llm_response(prompt))

حالا برنامه هر شهری که بخواهی کافیه نامش رو از دیکشنری صدا بزنی.

In [ ]:
# Print in Markdown format
display(Markdown(detailed_itinerary["Tokyo"]))

## تمرین برای خودت

کد پایین رو اصلاح کن تا برنامه دریافتی برای تک تک شهرهای پایین رو ببینی:
**گزینه‌ها:**
- Cape Town
- Istanbul
- New York
- Paris
- Rio de Janeiro
- Sydney
- Tokyo

In [ ]:
# نام شهر رو جایگزین کن
display(Markdown(detailed_itinerary["YOUR CITY HERE"]))

## شادباش! این هفته هم تموم شد 🎉🎉🎉

تمرینش کن. فقط تمرین کن و ایده‌هات رو امتحان کن. خراب کن. گیر کن. ادامه بده. پیشرفت می‌کنی